<a href="https://colab.research.google.com/github/fatihadr/UAS-NLP_Text-Classification/blob/main/UAS_NLP_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PROJECT UAS MATA KULIAH NATURAL LANGUAGE PROCESSING (NLP)**




*   Della Agustin (2215061116)
*   Syefara Raissa Ramdhan (2215061012)
*   Siti Fatiha Diza Rahman (2215061084)

**Link Streamlit:** https://text-classification-skripsi.streamlit.app/

*   Preprocessing : https://colab.research.google.com/drive/1-j50pbyVKeWOt4ZhQYlN3508_HAup7AW?usp=sharing
*   Modelling & Testing : https://colab.research.google.com/drive/1XHilCyYVt_vj5oAMTkXpGo2hAbeFtuTN?usp=sharing
*   (Resampling) Modelling & Testing : https://colab.research.google.com/drive/127mIg5wyjM8TsgefaVQ46Y3H3CogIx5l?usp=sharing







In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


# **Load Data**


In [ ]:
# Load CSV
eprint_df = pd.read_csv("/content/drive/MyDrive/Copy of eprint.csv", low_memory=False)
subject_df = pd.read_csv("/content/drive/MyDrive/Copy of eprint_subjects.csv")
# Tampilkan data awal
print("Eprint:")
print(eprint_df[['eprintid', 'title']].head())

print("\nSubject:")
print(subject_df.head())

Eprint:
   eprintid                                              title
0         4  Praktik Manajemen CSR (tanggung jawab sosial p...
1         6                                                NaN
2         8   \r\nOPTIMASI ALIRAN DAYA MENGGUNAKAN UNIFIED ...
3         9  KETAHANAN KULTIVAR CABAI MERAH (Capsicum annuu...
4        10  STUDI DAYA DUKUNG TANAH ORGANIK  MENGGUNAKAN M...

Subject:
   eprintid  pos subjects
0     33034    0      000
1     33072    0      000
2     54370    0      000
3     56064    0      000
4     56066    0      000


# **Exploratory Data Analysis (EDA)**

In [ ]:
eprint_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56974 entries, 0 to 56973
Data columns (total 91 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   eprintid                   56974 non-null  int64  
 1   rev_number                 56968 non-null  float64
 2   eprint_status              56968 non-null  object 
 3   userid                     56968 non-null  float64
 4   importid                   0 non-null      float64
 5   source                     0 non-null      float64
 6   dir                        56968 non-null  object 
 7   datestamp_year             51459 non-null  float64
 8   datestamp_month            51459 non-null  float64
 9   datestamp_day              51459 non-null  float64
 10  datestamp_hour             51459 non-null  float64
 11  datestamp_minute           51459 non-null  float64
 12  datestamp_second           51459 non-null  float64
 13  lastmod_year               56968 non-null  flo

In [ ]:
subject_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73721 entries, 0 to 73720
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   eprintid  73721 non-null  int64 
 1   pos       73721 non-null  int64 
 2   subjects  73721 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.7+ MB


In [ ]:
eprint_df.describe()

,eprintid,rev_number,userid,importid,source,datestamp_year,datestamp_month,datestamp_day,datestamp_hour,datestamp_minute,...,task_purpose,learning_level,gscholar_impact,gscholar_cluster,gscholar_datestamp_year,gscholar_datestamp_month,gscholar_datestamp_day,gscholar_datestamp_hour,gscholar_datestamp_minute,gscholar_datestamp_second
count,56974.000000,56968.000000,56968.000000,0.0,0.0,51459.000000,51459.000000,51459.000000,51459.000000,51459.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,41239.205023,15.888393,14629.573375,NaN,NaN,2019.009075,5.629938,17.006763,5.125712,29.614353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,28449.385166,11.116451,16424.633589,NaN,NaN,3.849370,3.499843,8.349943,2.967990,17.477021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,4.000000,1.000000,1.000000,NaN,NaN,2014.000000,1.000000,1.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,15579.250000,10.000000,73.000000,NaN,NaN,2015.000000,2.000000,10.000000,3.000000,14.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,30884.500000,13.000000,5074.500000,NaN,NaN,2018.000000,5.000000,18.000000,5.000000,30.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,68130.750000,19.000000,32427.250000,NaN,NaN,2022.000000,8.000000,24.000000,7.000000,45.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,87743.000000,257.000000,45731.000000,NaN,NaN,2025.000000,12.000000,31.000000,23.000000,59.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
subject_df.describe()

,eprintid,pos
count,73721.000000,73721.000000
mean,42836.036408,0.359870
std,29099.220452,0.625865
min,4.000000,0.000000
25%,16218.000000,0.000000
50%,32070.000000,0.000000
75%,70997.000000,1.000000
max,87743.000000,10.000000


# **Gabungkan Data**

In [ ]:
# Gabungkan berdasarkan eprintid
gabung_df = pd.merge(eprint_df[['eprintid', 'title']], subject_df[['eprintid', 'subjects']], on='eprintid')

# Tampilkan data akhir
print(gabung_df)


       eprintid                                              title subjects
0             4  Praktik Manajemen CSR (tanggung jawab sosial p...       HD
1             4  Praktik Manajemen CSR (tanggung jawab sosial p...     HD28
2             8   \r\nOPTIMASI ALIRAN DAYA MENGGUNAKAN UNIFIED ...   ZA4050
3             9  KETAHANAN KULTIVAR CABAI MERAH (Capsicum annuu...       Q1
4             9  KETAHANAN KULTIVAR CABAI MERAH (Capsicum annuu...    QH301
...         ...                                                ...      ...
73716     87740  PENGGUNAAN MEDIA AUTENTIK MAJALAH VOGUE FRANCE...      300
73717     87740  PENGGUNAAN MEDIA AUTENTIK MAJALAH VOGUE FRANCE...      370
73718     87741  ANALISIS KELAYAKAN USAHA PRODUK TELUR ASIN AYA...      630
73719     87742  ANALISIS STRATEGI PEMASARAN PRODUK\r\nTELUR AS...      630
73720     87743                                                NaN      320

[73721 rows x 3 columns]


# **Pre-Processing & Data Cleansing**


In [ ]:
# Menghapus baris yang title atau subjects-nya kosong
gabung_df.dropna(subset=['title', 'subjects'], inplace=True)
gabung_df

,eprintid,title,subjects
0,4,Praktik Manajemen CSR (tanggung jawab sosial p...,HD
1,4,Praktik Manajemen CSR (tanggung jawab sosial p...,HD28
2,8,\r\nOPTIMASI ALIRAN DAYA MENGGUNAKAN UNIFIED ...,ZA4050
3,9,KETAHANAN KULTIVAR CABAI MERAH (Capsicum annuu...,Q1
4,9,KETAHANAN KULTIVAR CABAI MERAH (Capsicum annuu...,QH301
...,...,...,...
73715,87739,PENGEMBANGAN MEDIA VIDEO TRANSKRIP SUPER FRANÇ...,440
73716,87740,PENGGUNAAN MEDIA AUTENTIK MAJALAH VOGUE FRANCE...,300
73717,87740,PENGGUNAAN MEDIA AUTENTIK MAJALAH VOGUE FRANCE...,370
73718,87741,ANALISIS KELAYAKAN USAHA PRODUK TELUR ASIN AYA...,630


In [ ]:
# Hapus baris yang nilai 'subjects'-nya bukan angka
# Convert 'subjects' to string type first to use .str accessor safely
gabung_df['subjects'] = gabung_df['subjects'].astype(str)
gabung_df = gabung_df[gabung_df['subjects'].str.isnumeric()].copy()

# Ubah kolom 'subjects' jadi tipe integer
gabung_df['subjects'] = gabung_df['subjects'].astype(int)

# Cek hasil akhir
gabung_df

,eprintid,title,subjects
9682,8282,PERAN GABUNGA KELOMPOK TANI (GAPOKTAN) BAKTI M...,630
13203,11260,ANALYS QUALITY CONTROL PRODUCT AT LIMITED PART...,300
13204,11260,ANALYS QUALITY CONTROL PRODUCT AT LIMITED PART...,350
13206,11261,THE ROLE OF INTERPERSONAL COMMUNICATION IN THE...,300
13207,11261,THE ROLE OF INTERPERSONAL COMMUNICATION IN THE...,302
...,...,...,...
73715,87739,PENGEMBANGAN MEDIA VIDEO TRANSKRIP SUPER FRANÇ...,440
73716,87740,PENGGUNAAN MEDIA AUTENTIK MAJALAH VOGUE FRANCE...,300
73717,87740,PENGGUNAAN MEDIA AUTENTIK MAJALAH VOGUE FRANCE...,370
73718,87741,ANALISIS KELAYAKAN USAHA PRODUK TELUR ASIN AYA...,630


In [ ]:
# Fungsi untuk membersihkan teks
def clean_text(text):
    text = str(text).lower()                         # lowercase
    text = re.sub(r'[^a-z0-9\s]', '', text)          # hapus tanda baca/simbol
    text = re.sub(r'\s+', ' ', text).strip()         # hapus spasi ganda
    return text

# Terapkan ke kolom judul
gabung_df['title'] = gabung_df['title'].apply(clean_text)
gabung_df

,eprintid,title,subjects
9682,8282,peran gabunga kelompok tani gapoktan bakti mak...,630
13203,11260,analys quality control product at limited part...,300
13204,11260,analys quality control product at limited part...,350
13206,11261,the role of interpersonal communication in the...,300
13207,11261,the role of interpersonal communication in the...,302
...,...,...,...
73715,87739,pengembangan media video transkrip super frana...,440
73716,87740,penggunaan media autentik majalah vogue france...,300
73717,87740,penggunaan media autentik majalah vogue france...,370
73718,87741,analisis kelayakan usaha produk telur asin aya...,630


In [ ]:
#stemming
stemmer = StemmerFactory().create_stemmer()
gabung_df['title'] = gabung_df['title'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))
gabung_df

,eprintid,title,subjects
9682,8282,peran gabunga kelompok tani gapoktan bakti mak...,630
13203,11260,analys quality control product at limited part...,300
13204,11260,analys quality control product at limited part...,350
13206,11261,the role of interpersonal communication in the...,300
13207,11261,the role of interpersonal communication in the...,302
...,...,...,...
73715,87739,kembang media video transkrip super franais ba...,440
73716,87740,guna media autentik majalah vogue france untuk...,300
73717,87740,guna media autentik majalah vogue france untuk...,370
73718,87741,analisis layak usaha produk telur asin ayam la...,630


In [ ]:
gabung_df['subjects'].value_counts().sort_index()


,count
subjects,
0,1082
1,642
2,6
3,97
4,137
...,...
909,4
910,79
920,2


# **Labeling**

In [ ]:
# Labeling
gabung_df['label'] = (gabung_df['subjects'] // 100) * 100
gabung_df['label'].value_counts().sort_index()



,count
label,
0,2372
100,171
200,24
300,15971
400,992
500,5005
600,8167
700,594
800,21


In [ ]:
gabung_df

,eprintid,title,subjects,label
9682,8282,peran gabunga kelompok tani gapoktan bakti mak...,630,600
13203,11260,analys quality control product at limited part...,300,300
13204,11260,analys quality control product at limited part...,350,300
13206,11261,the role of interpersonal communication in the...,300,300
13207,11261,the role of interpersonal communication in the...,302,300
...,...,...,...,...
73715,87739,kembang media video transkrip super franais ba...,440,400
73716,87740,guna media autentik majalah vogue france untuk...,300,300
73717,87740,guna media autentik majalah vogue france untuk...,370,300
73718,87741,analisis layak usaha produk telur asin ayam la...,630,600


# **Simpan File dalam bentuk CSV**

File yang telah melalui tahap preprocessing, seperti dat cleansing, stemming, dan pelabelan, kemudian disimpan dalam format CSV untuk digunakan pada tahap pemodelan. Tujuannya adalah agar proses pemodelan dapat dilakukan tanpa harus mengulang tahapan preprocessing, sehingga lebih efisien dalam penggunaan waktu

In [ ]:
gabung_df.to_csv("hasil_preprocessing.csv", index=False)
